# Preparing the 2015 and 2016 Freddie Mac Dataset

This notebook transforms raw mortgage loan data from 2015 and 2016 into a structured format suitable for machine learning. It performs archive extraction, data validation, placeholder cleaning, default outcome labeling, temporal filtering, and balanced sampling. The result is a cleaned and labeled dataset that enables robust downstream modeling for credit risk prediction.

Note: This script is intended for academic reference only.

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta

import zipfile

import os

def extract_zip_files(zip_dir, extract_to_dir):
    
    if not os.path.exists(extract_to_dir):
        os.makedirs(extract_to_dir)
        print(f"Created extraction directory: {extract_to_dir}")

    for item in os.listdir(zip_dir):
        full_path = os.path.join(zip_dir, item)
        if item.endswith(".zip"):
            try:
                with zipfile.ZipFile(full_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_to_dir)
                print(f"Successfully extracted {item} to {extract_to_dir}")
            except Exception as e:
                print(f"Error: An unexpected error occurred.")
        else:
            print(f"Skipping non-zip file: {item}")

def load_freddie_mac_data(year_quarter_str, data_type, base_path):
    
    file_prefix = 'origdata' if data_type == 'origination' else 'svcgdata'
    file_name = f"{file_prefix} {year_quarter_str}.csv"
    full_file_path = os.path.join(base_path, file_name)

    if not os.path.exists(full_file_path):
        print(f"File not found: {full_file_path}. Skipping.")
        return None
    try:
        df = pd.read_csv(full_file_path, low_memory=False)
        print(f"Successfully loaded {data_type} data from {file_name}.")
        return df
    except Exception as e:
        print(f"Error: Error loading {data_type} data from {full_file_path}: {e}")
        return None

def handle_freddie_mac_placeholders(data_df, columns_to_handle):

    processed_data = data_df.copy()
    
    placeholder_map = {
        'fico': 9999,
        'flag_fthb': '9',
        'mi_pct': 999,
        'cnt_units': 99,
        'occpy_sts': '9',
        'dti': 999,
        'ltv': 999,
        'channel': '9',
        'prop_type': '99',
        'loan_purpose': '9',
        'cnt_borr': 99
    }

    for col in columns_to_handle:
        if col in processed_data.columns:
            current_dtype = processed_data[col].dtype
            
            if pd.api.types.is_numeric_dtype(current_dtype):
                processed_data[col] = pd.to_numeric(processed_data[col], errors='coerce')
                placeholder = placeholder_map.get(col, np.nan)
                processed_data[col] = processed_data[col].replace(placeholder, np.nan)
            else:
                processed_data[col] = processed_data[col].astype(str)
                placeholder = str(placeholder_map.get(col, ''))
                processed_data[col] = processed_data[col].replace(placeholder, np.nan)
            
            print(f"Replaced placeholder in column '{col}'")

    if 'flag_sc' in processed_data.columns and 'flag_sc' in columns_to_handle:
        processed_data['flag_sc'] = processed_data['flag_sc'].fillna('N').astype(str)
    
    if 'flag_fthb' in processed_data.columns and 'flag_fthb' in columns_to_handle:
        processed_data['flag_fthb'] = processed_data['flag_fthb'].fillna('N').astype(str)
    
    if 'occpy_sts' in processed_data.columns and 'occpy_sts' in columns_to_handle:
        processed_data['occpy_sts'] = processed_data['occpy_sts'].fillna('U').astype(str)
        
    return processed_data

def calculate_d_timer(svcg_cycle_str, reference_date_str='201502'):

    if pd.isna(svcg_cycle_str) or svcg_cycle_str == 'N_A':
        return 1000
    
    svcg_cycle_str = str(svcg_cycle_str)
    
    if not svcg_cycle_str.isdigit() or len(svcg_cycle_str) != 6:
        return 1000
    
    try:
        current_date = datetime.strptime(svcg_cycle_str, "%Y%m")
        base_date = datetime.strptime(reference_date_str, "%Y%m")
        r = relativedelta(current_date, base_date)
        return r.months + (12 * r.years)
    except ValueError:
        return 1000

def process_performance_for_defaults(performance_df, observation_cutoff_date):
    
    if performance_df is None or performance_df.empty:
        return pd.DataFrame({'id_loan': [], 'loan_defaulted': [], 'svcg_cycle': []})

    performance_df = performance_df.copy()
    performance_df['id_loan'] = performance_df['id_loan'].astype(str)
    
    if 'svcg_cycle' in performance_df.columns:
        performance_df['svcg_cycle'] = performance_df['svcg_cycle'].astype(str)
        performance_df['svcg_cycle'] = performance_df['svcg_cycle'].apply(
            lambda x: x if len(str(x)) == 6 and str(x).isdigit() else 'N_A'
        )
    else:
        performance_df['svcg_cycle'] = 'N_A'
    
    print(f"Filtering performance data to only include data up to {observation_cutoff_date}")
    
    valid_cycles = performance_df['svcg_cycle'] != 'N_A'
    performance_df_filtered = performance_df[valid_cycles].copy()
    cutoff_filter = performance_df_filtered['svcg_cycle'] <= observation_cutoff_date
    performance_df_filtered = performance_df_filtered[cutoff_filter].copy()
    
    print(f"After filtering: {len(performance_df_filtered)} performance records (from {len(performance_df)} original)")
    
    if performance_df_filtered.empty:
        print("Warning: No performance data available within observation period")
        return pd.DataFrame({'id_loan': [], 'loan_defaulted': [], 'svcg_cycle': []})
    
    zero_bal_defaults = pd.DataFrame()
    if 'cd_zero_bal' in performance_df_filtered.columns:
        zero_bal_defaults = performance_df_filtered[
            (performance_df_filtered['cd_zero_bal'] == 3) | 
            (performance_df_filtered['cd_zero_bal'] == 6) | 
            (performance_df_filtered['cd_zero_bal'] == 9)
        ]
    
    performance_df_filtered.loc[:, 'delq_sts_numeric'] = performance_df_filtered['delq_sts'].replace('R', '-1', regex=False)
    performance_df_filtered.loc[:, 'delq_sts_numeric'] = pd.to_numeric(performance_df_filtered['delq_sts_numeric'], errors='coerce')
    initial_rows = len(performance_df_filtered)
    performance_df_filtered = performance_df_filtered.dropna(subset=['delq_sts_numeric']).reset_index(drop=True)
    
    if len(performance_df_filtered) < initial_rows:
        print(f"Removed {initial_rows - len(performance_df_filtered)} rows with invalid 'delq_sts'.")
    
    latest_performance = performance_df_filtered.sort_values('svcg_cycle', ascending=False).drop_duplicates('id_loan', keep='first')
    
    delq_defaults = performance_df_filtered[
        (performance_df_filtered['delq_sts_numeric'] == -1) | 
        (performance_df_filtered['delq_sts_numeric'] >= 3)
    ]
    
    all_defaults = pd.concat([delq_defaults, zero_bal_defaults]).drop_duplicates('id_loan', keep='first')
    defaults_flag = pd.DataFrame({'id_loan': all_defaults['id_loan'].unique(), 'loan_defaulted': 1})
    
    defaults_flag = pd.merge(
        defaults_flag, 
        latest_performance[['id_loan', 'svcg_cycle']], 
        on='id_loan', 
        how='outer'
    )
    
    print(f"Identified {len(defaults_flag)} defaulted loans within observation period (up to {observation_cutoff_date})")
    return defaults_flag

def filter_loans_with_sufficient_history(merged_data, min_months=12):
    
    print("Filtering loans with sufficient history.")
    
    if 'd_timer' not in merged_data.columns:
        print("Warning: 'd_timer' column not found, cannot filter by loan history")
        return merged_data
    
    sufficient_history = merged_data[merged_data['d_timer'] >= min_months].copy()
    print(f"Filtered from {len(merged_data)} to {len(sufficient_history)} loans with ≥{min_months} months OBSERVED history")
    
    return sufficient_history

def filter_for_complete_time_series(performance_df, merged_data, observation_cutoff_date, min_months_observed=6):

    print("Filtering for loans with sufficient observed time series.")
    
    if performance_df is None or performance_df.empty:
        print("Performance data is empty, cannot filter for time series completeness")
        return merged_data
    
    performance_df = performance_df.copy()
    performance_df['svcg_cycle'] = performance_df['svcg_cycle'].astype(str)
    valid_cycles = (performance_df['svcg_cycle'] != 'N_A') & (performance_df['svcg_cycle'] <= observation_cutoff_date)
    performance_filtered = performance_df[valid_cycles].copy()
    loan_observation_counts = performance_filtered.groupby('id_loan').size()
    sufficient_loans = loan_observation_counts[loan_observation_counts >= min_months_observed].index
    complete_loans_data = merged_data[merged_data['id_loan'].isin(sufficient_loans)].copy()
    
    print(f"Filtered from {len(merged_data)} to {len(complete_loans_data)} loans with ≥{min_months_observed} observed months")
    return complete_loans_data

def balanced_downsampling(merged_data, target_sample_size=40000):

    print("Applying balanced downsampling...")
    
    if len(merged_data) <= target_sample_size:
        print(f"Data size ({len(merged_data)}) is already smaller than target size ({target_sample_size}). No downsampling needed.")
        return merged_data
    
    target_default_rate = 0.06
    default_col = 'loan_defaulted'
    defaults_df = merged_data[merged_data[default_col] == 1]
    non_defaults_df = merged_data[merged_data[default_col] == 0]
    num_defaults = int(target_sample_size * target_default_rate)
    num_non_defaults = target_sample_size - num_defaults
    
    if len(defaults_df) < num_defaults:
        print(f"Warning: Only {len(defaults_df)} defaulted loans available, using all of them.")
        num_defaults = len(defaults_df)
        num_non_defaults = target_sample_size - num_defaults

        if target_sample_size > 0:
            actual_achieved_rate = num_defaults / target_sample_size
            print(f"Actual achieved default rate due to limited defaults: {actual_achieved_rate*100:.2f}%")
        else:
            actual_achieved_rate = 0
    else:
        actual_achieved_rate = target_default_rate

    if len(non_defaults_df) < num_non_defaults:
        print(f"Warning: Only {len(non_defaults_df)} non-defaulted loans available, using all of them.")
        num_non_defaults = len(non_defaults_df)
        num_defaults = target_sample_size - num_non_defaults
        if target_sample_size > 0:
            actual_achieved_rate = num_defaults / target_sample_size
            print(f"Actual achieved default rate due to limited non-defaults: {actual_achieved_rate*100:.2f}%")
        else:
            actual_achieved_rate = 0

    sampled_defaults = defaults_df.sample(n=num_defaults, random_state=42)
    sampled_non_defaults = non_defaults_df.sample(n=num_non_defaults, random_state=42)
    downsampled_data = pd.concat([sampled_defaults, sampled_non_defaults]).sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"Downsampled to {len(downsampled_data)} samples with {actual_achieved_rate*100:.2f}% defaults ({len(sampled_defaults)} defaulted loans).")
    return downsampled_data

def process_freddie_mac_quarterly_data(year, quarter, extracted_csv_dir, observation_cutoff_date, target_sample_size=40000):

    print(f"Starting data processing for {year}-{quarter}.")
    print(f"Observation cutoff date: {observation_cutoff_date}")
    
    current_year_extracted_dir = os.path.join(extracted_csv_dir, str(year))

    year_quarter_str = f"{year}-{quarter}"
    origination_df = load_freddie_mac_data(year_quarter_str, 'origination', current_year_extracted_dir)
    performance_df = load_freddie_mac_data(year_quarter_str, 'performance', current_year_extracted_dir)

    if origination_df is None or performance_df is None:
        print(f"Error: Failed to load essential data for {year_quarter_str}. Skipping further processing for this quarter.")
        return pd.DataFrame()

    origination_df.loc[:, 'id_loan'] = origination_df['id_loan'].astype(str)
    performance_df.loc[:, 'id_loan'] = performance_df['id_loan'].astype(str)
    defaults_flag_df = process_performance_for_defaults(performance_df, observation_cutoff_date)
    
    orig_cols_needed = [
        'id_loan', 'fico', 'flag_fthb', 'mi_pct', 'cnt_units', 'occpy_sts',
        'dti', 'ltv', 'channel', 'prod_type', 'prop_type', 'loan_purpose',
        'cnt_borr', 'seller_name', 'flag_sc', 'servicer_name', 'orig_upb',
        'dt_first_pi', 'orig_loan_term', 'zipcode', 'loan_age'
    ]
    
    existing_orig_cols_needed = [col for col in orig_cols_needed if col in origination_df.columns]
    origination_df_filtered = origination_df[existing_orig_cols_needed].copy()
     
    if 'dt_first_pi' in origination_df_filtered.columns:
        origination_df_filtered.loc[:, 'dt_first_pi_str'] = origination_df_filtered['dt_first_pi'].astype(str)
        initial_rows_before_date_filter = len(origination_df_filtered)
        valid_date_rows = origination_df_filtered['dt_first_pi_str'].str.len() == 6
        origination_df_filtered = origination_df_filtered[valid_date_rows].reset_index(drop=True)
        
        if len(origination_df_filtered) < initial_rows_before_date_filter:
            print(f"Dropped {initial_rows_before_date_filter - len(origination_df_filtered)} rows with invalid 'dt_first_pi' string length (not YYYYMM).")

        if not origination_df_filtered.empty:
            origination_df_filtered.loc[:, 'year'] = origination_df_filtered['dt_first_pi_str'].str[0:4].astype(int)
            origination_df_filtered.loc[:, 'month'] = origination_df_filtered['dt_first_pi_str'].str[4:6].astype(int)
            
            initial_rows_before_month_filter = len(origination_df_filtered)
            valid_month_rows = (origination_df_filtered['month'] >= 1) & (origination_df_filtered['month'] <= 12)
            origination_df_filtered = origination_df_filtered[valid_month_rows].reset_index(drop=True)
            
            if len(origination_df_filtered) < initial_rows_before_month_filter:
                print(f"Dropped {initial_rows_before_month_filter - len(origination_df_filtered)} rows with invalid month values in 'dt_first_pi'.")

            origination_df_filtered.drop(columns=['dt_first_pi_str'], inplace=True)
    
    orig_cols_for_placeholder_map = [
        'fico', 'flag_fthb', 'mi_pct', 'cnt_units', 'occpy_sts', 'dti', 'ltv',
        'channel', 'prop_type', 'loan_purpose', 'cnt_borr'
    ]
    
    if 'flag_sc' in existing_orig_cols_needed:
        orig_cols_for_placeholder_map.append('flag_sc')

    actual_cols_for_placeholder = [col for col in orig_cols_for_placeholder_map if col in origination_df_filtered.columns]
    origination_df_filtered = handle_freddie_mac_placeholders(origination_df_filtered, actual_cols_for_placeholder)
    merged_data = pd.merge(origination_df_filtered, defaults_flag_df, on='id_loan', how='left')
    merged_data.loc[:, 'loan_defaulted'] = merged_data['loan_defaulted'].fillna(0).astype(int)
    merged_data.loc[:, 'svcg_cycle'] = merged_data['svcg_cycle'].fillna('N_A')
    
    print("Merged origination data with loan default flags and observed servicing cycle.")
    
    if 'svcg_cycle' in merged_data.columns:
        merged_data['d_timer'] = merged_data['svcg_cycle'].apply(
            lambda x: calculate_d_timer(x, reference_date_str='201502')
        )
        print("Added 'd_timer' feature measuring months since February 2015.")

    merged_data = filter_loans_with_sufficient_history(merged_data, min_months=6)
    merged_data = filter_for_complete_time_series(performance_df, merged_data, observation_cutoff_date, min_months_observed=6)
    
    if len(merged_data) > target_sample_size:
        merged_data = balanced_downsampling(merged_data, target_sample_size)

    if 'Unnamed: 0' in merged_data.columns:
        merged_data = merged_data.drop(columns=['Unnamed: 0'])
        
    merged_data.loc[:, 'id'] = np.arange(1, len(merged_data) + 1)

    desired_order_prefix = [
        'id', 'id_loan', 'year', 'month', 'svcg_cycle', 'd_timer',
        'fico', 'flag_fthb', 'mi_pct', 'cnt_units', 'occpy_sts', 'dti',
        'ltv', 'channel', 'prop_type', 'loan_purpose', 'cnt_borr', 'flag_sc',
        'orig_upb', 'loan_defaulted', 'orig_loan_term', 'seller_name', 'servicer_name', 'zipcode'
    ]
    final_cols = [col for col in desired_order_prefix if col in merged_data.columns]
    for col in merged_data.columns:
        if col not in final_cols:
            final_cols.append(col)

    merged_data = merged_data[final_cols].reset_index(drop=True)
    print(f"### Finished data processing for {year}-{quarter} ###")
    return merged_data

if __name__ == "__main__":
    
    pd.set_option('future.no_silent_downcasting', True)
    RAW_ZIP_DATA_DIR = '../data/raw_data'
    EXTRACTED_CSV_DATA_DIR = '../data/extracted_data'
    OUTPUT_PREPARED_DATA_DIR = '../data/prepared_data'
    TARGET_SAMPLE_SIZE_PER_QUARTER = 15000

    if not os.path.exists(OUTPUT_PREPARED_DATA_DIR):
        os.makedirs(OUTPUT_PREPARED_DATA_DIR)
        print(f"Created output directory for prepared data: {OUTPUT_PREPARED_DATA_DIR}")


    OBSERVATION_CUTOFFS = {
        2015: '201612', 
        2016: '201712' 
    }

    years_to_process = [2015, 2016]
    all_yearly_dataframes = {}

    for current_target_year in years_to_process:
        
        observation_cutoff = OBSERVATION_CUTOFFS[current_target_year]
        print(f"\nStarting overall data processing for {current_target_year}")
        print(f"### Observation cutoff: {observation_cutoff}.###")

        current_year_zip_dir = os.path.join(RAW_ZIP_DATA_DIR, str(current_target_year))
        current_year_extracted_dir = os.path.join(EXTRACTED_CSV_DATA_DIR, str(current_target_year))
        print(f"Extracting all zip files for {current_target_year} from {current_year_zip_dir} to {current_year_extracted_dir}...")
        extract_zip_files(current_year_zip_dir, current_year_extracted_dir)
        print(f"Finished initial extraction for {current_target_year}.\n")

        all_quarters_data_for_year = []

        for quarter_num in range(1, 5):
            current_quarter_str = f'Q{quarter_num}'
            print(f"\n### Processing {current_target_year}-{current_quarter_str} Data. ###")

            processed_quarter_data = process_freddie_mac_quarterly_data(
                current_target_year, current_quarter_str, EXTRACTED_CSV_DATA_DIR, 
                observation_cutoff, TARGET_SAMPLE_SIZE_PER_QUARTER
            )

            if not processed_quarter_data.empty:
                all_quarters_data_for_year.append(processed_quarter_data)
            else:
                print(f"No data processed for {current_target_year}-{current_quarter_str}.")

        df_prepared_for_year = pd.DataFrame()
        if all_quarters_data_for_year:
            print(f"Attempting to concatenate {len(all_quarters_data_for_year)} quarters of {current_target_year} data.")
            try:
                df_prepared_for_year = pd.concat(all_quarters_data_for_year, ignore_index=True)
                print(f"Successfully concatenated all {current_target_year} quarters. Total rows: {len(df_prepared_for_year)}")

                all_yearly_dataframes[current_target_year] = df_prepared_for_year

                output_filename = os.path.join(OUTPUT_PREPARED_DATA_DIR, f'prepared_{current_target_year}_full_year_data.csv')
                df_prepared_for_year.to_csv(output_filename, index=False)
                print(f"Saved prepared {current_target_year} full year data to '{output_filename}'")

                print(f"\n### Full Year {current_target_year} Data: Final Combined Inspection ###")
                print("Shape:", df_prepared_for_year.shape)
                print("Missing values (top 5 columns):\n", df_prepared_for_year.isnull().sum().nlargest(5))
                print("Default counts in full year data:\n", df_prepared_for_year['loan_defaulted'].value_counts(dropna=False))
                print("Unique loan IDs in full year data:", df_prepared_for_year['id_loan'].nunique())

            except Exception as e:
                print(f"Error concatenating {current_target_year} dataframes: {e}")
        else:
            print(f"Warning: No {current_target_year} dataframes were successfully processed for concatenation.")

    print(f"\n### Finished all data processing for years {years_to_process} ###")